In [127]:
import torch
import math

import matplotlib.pyplot as plt
import openturns as ot
import numpy as np

import torch.nn as nn

import random

import torch.nn.functional as F

In [182]:
class SocialSig(nn.Module):
    def __init__(self):
        super(SocialSig, self).__init__()
        # self.weight = torch.nn.Parameter(data=torch.Tensor(1, 1, 1, 1), requires_grad=True)
        self.weight = torch.nn.Parameter(torch.randn(()))
        
        self.coords = coords
        
#         self.weight.data.uniform_(-1, 1)


    def __train_krigging(self, points, coords, weight):
        '''
        Train the krigging model to predict all of the points that are between known points
        '''
        
        random.Random(weight).shuffle(points)
        
        coordinates_train = ot.Sample(coords)
        data_train = ot.Sample(points) # At 2000/1/2
        inputDimension = 2
        basis = ot.ConstantBasisFactory(inputDimension).build()
        covarianceModel = ot.SquaredExponential([1.]*inputDimension, [1.0])
        algo = ot.KrigingAlgorithm(coordinates_train, data_train, covarianceModel, basis)
        algo.run()
        result = algo.getResult()
        krigingMetamodel = result.getMetaModel()
        return [krigingMetamodel, points]
    
    
    def __make_blank_coord_grid(self, n_rows):
        '''
        Make a blank coordinate grid to fill in with real data later
        '''
        return [[x,y] for x in range(0, 4) for y in range(0,4)]
            

    def __make_real_data_set(self, blank_grid, points, coords, krigingMetamodel):
        '''
        Fill in the blank grid with real data.
        If the coordinate pair is in the list that has real data, give the coord real data.
        If not, krig it.
        '''
        
        to_return = [(points[coords.index(i)][0]) if i in coords else list(krigingMetamodel(i))[0] for i in blank_grid]
        to_return = torch.from_numpy(np.reshape(np.array(to_return), (1,1,4,4)))
        to_return = torch.tensor(to_return, dtype=torch.float)

        return to_return
    
    def process(self, x, coords, weight):
#         self.__points, self.__coords = self.__make_training_data(num_known_points, n_rows, seed)
        krig, points = self.__train_krigging(x, coords, self.weight)
        blank_grid = self.__make_blank_coord_grid(2)
        real_dataset = self.__make_real_data_set(blank_grid, points, coords, krig)
        
        return real_dataset

        
    
    def forward(self, x):
        
        print("WEIGHT: ", self.weight)
        
#         coords = [(list(np.random.randint(low = 0, high = math.sqrt(len(x)), size = 2))) for i in range(0, len(x))]        
#         self.process(x, coords, self.weight)
        return self.process(x, self.coords, self.weight) * self.weight

In [124]:
model = SocialSig()
model(make_example_data(14))

NameError: name 'weight' is not defined

In [125]:
# THIS IS WHAT YOU WILL USE TO CREATE YOUR INPUT LAYER--THIS SHOULD BE A LIST WITH 81 ELEMENTS (THE ROW IN THE DF)
def make_example_data(seed):
    '''
    Create the training dataset by randomly assigning coordiantes to the known data (i.e the data within a row)
    '''
    points = [[i] for i in list(np.random.randint(low = 0, high = 255, size = 4).flatten())]
#         coords = [(list(np.random.randint(low = 0, high = n_rows, size = 2))) for i in range(0, num_known_points)]
#     points = [[60], [48], [151], [52], [95], [135], [184], [199]]
#     random.Random(seed).shuffle(points)
#     coords = [[0, 2], [1, 0], [3, 1], [1, 0], [0, 3], [2, 0], [3, 0], [3, 2]]
    return points


make_example_data(14)

[[213], [88], [240], [240]]

In [126]:
# class SocialSig():
    
#     '''
#     Class to create the social signature image
#     '''
    
#     def __init__(self, num_known_points, n_rows, seed):
#         '''
#         Basic steps for class:
#             1. Randomly initalize 'weights' which I believe are actually the coords of the points
#             2. Train kriggin model to interpolate points in between
#             3. Predict what the points would be within a 224x224 matrix and output the resulting matrix
#         '''
#         self.__points, self.__coords = self.__make_training_data(num_known_points, n_rows, seed)
#         self.__krig = self.__train_krigging(self.__points, self.__coords)
#         self.__blank_grid = self.__make_blank_coord_grid(n_rows)
#         self.real_dataset = self.__make_real_data_set(self.__blank_grid, self.__points, self.__coords, self.__krig)
        
        
#     def __make_training_data(self, num_known_points, n_rows, seed):
#         '''
#         Create the training dataset by randomly assigning coordiantes to the known data (i.e the data within a row)
#         '''
# #         points = [[i] for i in list(np.random.randint(low = 0, high = 255, size = num_known_points).flatten())]
# #         coords = [(list(np.random.randint(low = 0, high = n_rows, size = 2))) for i in range(0, num_known_points)]
#         points = [[60], [48], [151], [52], [95], [135], [184], [199]]
#         random.Random(seed).shuffle(points)
#         coords = [[0, 2], [1, 0], [3, 1], [1, 0], [0, 3], [2, 0], [3, 0], [3, 2]]
#         return points, coords
    
    
#     def __train_krigging(self, points, coords):
#         '''
#         Train the krigging model to predict all of the points that are between known points
#         '''
#         coordinates_train = ot.Sample(coords)
#         data_train = ot.Sample(points) # At 2000/1/2
#         inputDimension = 2
#         basis = ot.ConstantBasisFactory(inputDimension).build()
#         covarianceModel = ot.SquaredExponential([1.]*inputDimension, [1.0])
#         algo = ot.KrigingAlgorithm(coordinates_train, data_train, covarianceModel, basis)
#         algo.run()
#         result = algo.getResult()
#         krigingMetamodel = result.getMetaModel()
#         return krigingMetamodel
    
    
#     def __make_blank_coord_grid(self, n_rows):
#         '''
#         Make a blank coordinate grid to fill in with real data later
#         '''
#         return [[x,y] for x in range(0, 4) for y in range(0,4)]
            

#     def __make_real_data_set(self, blank_grid, points, coords, krigingMetamodel):
#         '''
#         Fill in the blank grid with real data.
#         If the coordinate pair is in the list that has real data, give the coord real data.
#         If not, krig it.
#         '''
        
#         to_return = [(points[coords.index(i)][0]) if i in coords else list(krigingMetamodel(i))[0] for i in blank_grid]
#         to_return = torch.from_numpy(np.reshape(np.array(to_return), (1,1,4,4)))
#         to_return = torch.tensor(to_return, dtype=torch.float)

#         return to_return
        

In [183]:
# class TwoLayerNet(torch.nn.Module):
#     def __init__(self, D_in, H, D_out):
#         """
#         In the constructor we instantiate two nn.Linear modules and assign them as
#         member variables.
#         """
#         super(TwoLayerNet, self).__init__()
#         self.linear1 = torch.nn.Linear(D_in, H)
#         self.linear2 = torch.nn.Linear(H, D_out)

#     def forward(self, x):
#         """
#         In the forward function we accept a Tensor of input data and we must return
#         a Tensor of output data. We can use Modules defined in the constructor as
#         well as arbitrary operators on Tensors.
#         """
#         h_relu = self.linear1(x).clamp(min=0)
#         y_pred = self.linear2(h_relu)
#         return y_pred


class MakeCoords(nn.Module):
    def __init__(self):
        super(MakeCoords, self).__init__()
        # self.weight = torch.nn.Parameter(data=torch.Tensor(1, 1, 1, 1), requires_grad=True)
#         self.weight = torch.nn.Parameter(torch.randn(()))
        
#         self.weight.data.uniform_(-1, 1)

        self.x1 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        self.y1 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        
        self.x2 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        self.y2 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        
        self.x3 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        self.y3 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        
        self.x4 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        self.y4 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32))   
        
        self.x5 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        self.y5 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        
        self.x6 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        self.y6 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        
        self.x7 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        self.y7 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        
        self.x8 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 
        self.y8 = torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32))          
    
    def forward(self, x):
        
        
        
        coords = torch.tensor([[1*self.x1, 1*self.y1], [1*self.x2, 1*self.y2], [1*self.x3, 1*self.y3], \
                  [1*self.x4, 1*self.y4], [1*self.x5, 1*self.y5], [1*self.x6, 1*self.y6], \
                  [1*self.x7, 1*self.y7], [1*self.x8, 1*self.y8]])
                
#         print("WEIGHT: ", self.weight)
        
#         coords = [(list(np.random.randint(low = 0, high = math.sqrt(len(x)), size = 2))) for i in range(0, len(x))]        
# #         self.process(x, coords, self.weight)
        return coords





class TwoLayerNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.SocialSig = SocialSig()
        self.conv1 = nn.Conv2d(1, 4, 4, 1)
        self.linear1 = torch.nn.Linear(4, 1)
        self.linear2 = torch.nn.Linear(2, 1)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        coords = MakeCoords(x)
        sig = self.SocialSig(x, coords)
#         print(sig)
        sig = self.conv1(sig)
        sig = torch.flatten(sig, 1)
        sig = self.linear1(sig)#.clamp(min=0)
#         y_pred = self.linear2(sig)
        return sig

In [184]:
model = MakeCoords()
model(1)

tensor([[ 7.,  8.],
        [10.,  7.],
        [ 0.,  8.],
        [ 4., 10.],
        [10.,  6.],
        [ 9.,  4.],
        [ 8.,  7.],
        [10.,  0.]])

In [158]:
torch.nn.Parameter(torch.randn(())) 

Parameter containing:
tensor(-1.7541, requires_grad=True)

In [161]:
torch.nn.Parameter(torch.tensor(random.randint(0,10), dtype = torch.float32)) 

Parameter containing:
tensor(10., requires_grad=True)

In [152]:
# # N is batch size; D_in is input dimension;
# # H is hidden dimension; D_out is output dimension.
# N, D_in, H, D_out = 64, 1000, 100, 1

# # Create random Tensors to hold inputs and outputs
# x = torch.randn(N, D_in)
# y = torch.randn(N, D_out)

In [153]:
# x.shape

In [154]:
# y.shape

In [185]:
# Construct our model by instantiating the class defined above
model = TwoLayerNet()

TypeError: __init__() missing 1 required positional argument: 'coords'

In [156]:
# model()

In [157]:
x = make_example_data(14)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(5):
    # Forward pass: Compute predicted y by passing x to the model
    print(x)
    y_pred = model(x)
    
    print("PRED: ", y_pred)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print("\n")

[[183], [203], [142], [132]]
WEIGHT:  Parameter containing:
tensor(0.5752, requires_grad=True)
PRED:  tensor([[-4.4638]], grad_fn=<AddmmBackward>)


[[132], [203], [183], [142]]
WEIGHT:  Parameter containing:
tensor(0.1163, requires_grad=True)
PRED:  tensor([[208.2670]], grad_fn=<AddmmBackward>)


[[142], [203], [132], [183]]
WEIGHT:  Parameter containing:
tensor(-4764.9541, requires_grad=True)
PRED:  tensor([[-2.9707e+11]], grad_fn=<AddmmBackward>)


[[183], [203], [142], [132]]
WEIGHT:  Parameter containing:
tensor(2.3707e+17, requires_grad=True)
PRED:  tensor([[nan]], grad_fn=<AddmmBackward>)


[[132], [203], [183], [142]]
WEIGHT:  Parameter containing:
tensor(nan, requires_grad=True)
PRED:  tensor([[nan]], grad_fn=<AddmmBackward>)




/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/ipykernel_launcher.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [103]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.


In [ ]:
'''
Make a layer that has a x trainabale params as the coordinates
    -- THIS LAYER NEEDS TO HAVE A FORWARD PASS
    (https://discuss.pytorch.org/t/pytorch-equivalent-of-keras/29412)
'''
 